In [1]:
import pandas as pd

In [2]:
f=open("C:/Users/PC/Desktop/Settlements/1_11_2016_settlements.txt", 'r')
f1 = f.readlines()


In [3]:
f1[7]

'FEB17     1.494     1.496B    1.480     1.496B    1.499   -.016           9      1.515         244         340\n'

In [4]:
import re
text = [re.sub("\s+", ",", f.strip()).split(',') for f in f1]
import csv
with open('C:/Users/PC/Desktop/output.csv','w',newline='') as result_file:
    wr = csv.writer(result_file)
    wr.writerows(text)

In [5]:
text[0]

['FINAL',
 'PRE-CLEARING',
 'PRICES',
 'AS',
 'OF',
 '11/01/16',
 '06:00',
 'PM',
 '(CST)']

In [6]:
import os
lst =os.listdir('C:/Users/PC/Desktop/Settlements')

In [7]:
lst

['10_10_2017_ags_settlements.txt',
 '10_11_2016_ags_settlements.txt',
 '10_11_2017_ags_settlements.txt',
 '10_1_2017_ags_settlements.txt',
 '10_1_2018_ags_settlements.txt',
 '10_1_2018_nymex_settlements.txt',
 '10_1_2019_ags_settlements.txt',
 '10_1_2019_eonly_settlements.txt',
 '10_1_2019_nymex_settlements.txt',
 '10_2_2017_ags_settlements.txt',
 '10_3_2017_ags_settlements.txt',
 '10_4_2017_ags_settlements.txt',
 '10_4_2019_ags_settlements.txt',
 '10_4_2019_int_settlements.txt',
 '10_5_2017_ags_settlements.txt',
 '10_5_2017_int_settlements.txt',
 '10_5_2018_ags_settlements.txt',
 '10_6_2019_ags_settlements.txt',
 '10_7_2017_ags_settlements.txt',
 '10_8_2017_ags_settlements.txt',
 '10_8_2018_ags_settlements.txt',
 '10_9_2018_ags_settlements.txt',
 '11_10_2016_settlements.txt',
 '11_11_2016_ags_settlements.txt',
 '11_12_2018_ags_settlements.txt',
 '11_1_2017_ags_settlements.txt',
 '11_1_2018_ags_settlements.txt',
 '11_3_2019_ags_settlements.txt',
 '11_4_2017_ags_settlements.txt',
 '11_4

In [8]:
archs = pd.DataFrame([l.split('_') for l in lst])

In [9]:
archs[0].dtype

dtype('O')

In [10]:
archs[[0,1,2]] = archs[[0,1,2]].astype(int)

In [11]:
archs[2].dtype

dtype('int32')

In [12]:
arches = archs.sort_values(by=[2,1,0])

In [13]:
list(set(arches[3]))

['settlements.txt', 'int', 'ags', 'eonly', 'nymex']

In [14]:
spoo = arches[arches[3] == 'eonly']

In [19]:
doc = []
with open('C:/Users/PC/Desktop/Settlements/' + lst[spoo.index[0]]) as f:
    for row in f:
        doc.append(row)

In [27]:
doc[1:10]

['MTH/                 ---- DAILY ---                        PT                     -------  PRIOR  DAY  -------\n',
 'STRIKE     OPEN      HIGH      LOW       LAST      SETT    CHGE     EST.VOL       SETT         VOL         INT\n',
 'ES   E-MINI S&P 500 FUTURES\n',
 'DEC16   2205.00   2212.00   2200.25   2211.50   2210.00   +5.75     1475806    2204.25     1663731     2923523\n',
 'MAR17   2199.75   2206.50   2195.00   2206.25   2204.75   +5.75      104724    2199.00       53305      113530\n',
 'JUN17   2193.00   2201.00   2190.00A  2200.25B  2199.25   +5.75        1168    2193.50         524        5019\n',
 'SEP17   2191.00   2192.00   2191.00   2192.00   2194.00   +5.75           4    2188.25           3          30\n',
 'DEC17      ----      ----      ----      ----   2189.25   +5.75                2183.50                      25\n',
 'TOTAL                                                               EST.VOL                 VOLUME    OPEN INT\n']

In [32]:
ew = [d.startswith('EW') for d in doc]
ez = [d.startswith('EZ') for d in doc]
ew_index = pd.DataFrame(doc)[ew].index
ez_index = pd.DataFrame(doc)[ez].index

In [33]:
ew_index

Int64Index([2021, 2276, 2531, 2786, 3027, 3268, 3511, 3766, 4021, 4276, 4517,
            4758],
           dtype='int64')

In [29]:
text = [re.sub("\s+", ",", d.strip()).split(',') for d in doc]

In [53]:
df = pd.DataFrame(text)[[0,8]]
df.columns = ['strike', 'oi']

In [54]:
last_ind = ew_index[0] 
while df.strike[last_ind + 1].isnumeric():
    last_ind += 1

In [57]:
df[ew_index[0]:last_ind][1:].dropna()

,strike,oi
2049,1480,9
2052,1510,5
2056,1540,12
2057,1550,12
2060,1575,7
2069,1650,1
2074,1690,11
2077,1720,13
2085,1775,19
2100,1850,11


In [50]:
df = pd.DataFrame(text).iloc[ew_index[0]:ew_index[1]][[0,8]][1:]
df.columns = ['strike', 'oi']
df.dropna()

,strike,oi
2049,1480,9
2052,1510,5
2056,1540,12
2057,1550,12
2060,1575,7
2069,1650,1
2074,1690,11
2077,1720,13
2085,1775,19
2100,1850,11
